In [1]:
import sys
import os.path as path
import matplotlib.pyplot as plt
from scipy.signal import remez
import numpy as np
from scipy.stats import zscore

import pandas as pd

In [2]:
%config Completer.use_jedi=False

In [3]:
sys.path.append(path.abspath('../..')) 
import diss3_code.database.get_db_data as gdd
import diss3_code.signal.smooth_component_analysis as sca

In [4]:

data = gdd.get_db_data()
data.get_participants()

[(1001, 101),
 (1001, 102),
 (1001, 103),
 (1003, 108),
 (1003, 109),
 (1004, 110),
 (1004, 111),
 (1004, 112),
 (1005, 113),
 (1005, 114),
 (1005, 115),
 (1006, 116),
 (1006, 117),
 (1006, 118),
 (1007, 119),
 (1007, 120),
 (1007, 121),
 (1008, 122),
 (1008, 123),
 (1008, 124),
 (1009, 125),
 (1009, 126),
 (1009, 127),
 (1010, 128),
 (1010, 129),
 (1010, 130),
 (1011, 131),
 (1011, 132),
 (1011, 133),
 (1012, 134),
 (1012, 135),
 (1012, 136),
 (1013, 137),
 (1013, 138),
 (1013, 139),
 (1014, 140),
 (1014, 141),
 (1014, 142),
 (1015, 144),
 (1015, 145),
 (1016, 146),
 (1016, 147),
 (1016, 148),
 (1017, 149),
 (1017, 151),
 (1019, 155),
 (1019, 156),
 (1019, 157),
 (1020, 158),
 (1020, 159),
 (1020, 160),
 (1022, 164),
 (1022, 165),
 (1022, 166),
 (1023, 167),
 (1023, 168),
 (1023, 169),
 (1024, 170),
 (1024, 171),
 (1024, 172),
 (1025, 173),
 (1025, 174),
 (1025, 175),
 (1026, 176),
 (1026, 177),
 (1026, 178),
 (1027, 179),
 (1027, 180),
 (1027, 181),
 (1028, 182),
 (1028, 183),
 (1028

In [5]:
file_data  = pd.read_json('test_json.zip', compression='infer')

In [6]:
file_data.columns

Index(['group', 'participant', 'session', 'path', 'data'], dtype='object')

### verify that all 141 valid participants are available in both forms
we still need to verify regarding specific series if we want more than just the FS series

In [27]:
db_participants = set(tuple(t) for t in data.get_participants())
nir_excel_participants = set(tuple(t) for t in file_data[['group','participant']].itertuples(index=False))
                            
db_participants - nir_excel_participants, nir_excel_participants - db_participants    

(set(), set())

In [28]:
len(db_participants), len(nir_excel_participants), len(db_participants & nir_excel_participants)

(141, 141, 141)

In [43]:
groups=set(t[0] for t in db_participants)
pars=[t[1] for t in sorted(db_participants)]


In [70]:
pars[:3]

[101, 102, 103]

In [67]:
data_dir = path.abspath('../../../data/time_series_linear_interpolations/')
path.exists(data_dir), data_dir

(True,
 '/home/siegman/Documents/Shahar/BarIlan/Dissertation3/data/time_series_linear_interpolations')

In [45]:
freestyle_raw = data.get_signal_by_series_type_and_interp_type(pars, 4,3)
next(freestyle_raw)

(101,
 array([[  4.     ,   0.97448],
        [  4.5    ,   0.98139],
        [  5.     ,   0.99647],
        ...,
        [255.     ,   0.91344],
        [255.5    ,   0.91352],
        [256.     ,   0.89942]]))

In [85]:
rows = []
freestyle_raw_2hz = data.get_signal_by_series_type_and_interp_type(pars, 4,3)

for (gr, par1), (par2,ts_2hz) in zip(
        sorted(db_participants), 
        freestyle_raw_2hz, 
    ):
    if par1==par2 :
        rows.append([gr, par1, ts_2hz.T])
    else:
        print("skipped {}".format(par1))
        break

In [86]:
freestyle_raw_df = pd.DataFrame(data=rows,columns='group,participant,signal_2hz'.split(','))

In [87]:
freestyle_raw_df.head()

,group,participant,signal_2hz
0,1001,101,"[[4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0,..."
1,1001,102,"[[4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0,..."
2,1001,103,"[[4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0,..."
3,1003,108,"[[3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5,..."
4,1003,109,"[[3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5,..."


In [88]:
freestyle_raw_df[freestyle_raw_df.signal_2hz.map(lambda l: l.shape[-1]<50)]

,group,participant,signal_2hz
55,1023,168,[]


In [91]:
t,x =freestyle_raw_df.iloc[0,-1]
t.shape, x.shape
np.vstack([t,x]).shape

(2, 505)

In [103]:
h = remez(25, [0., 0.1, 0.15, 1.],[1., 0.],fs =2.)

def lowpass(sig):
    if sig.ndim < 2:
        return np.array([])
    t,x = sig
    y = np.convolve(x,h, mode='valid')
    t = t[12:-12]
    return np.vstack([t,y])
    

In [104]:
freestyle_raw_df['remez_01_015']=freestyle_raw_df.signal_2hz.map(lowpass)

In [105]:
freestyle_raw_df.to_json(path.join(data_dir,'freestyle_raw.json'))

## encode baseline signals as dataframe

In [106]:
rows = []
baseline_raw = data.get_signal_by_series_type_and_interp_type(pars, 2,3)
for (gr, par1), (par2,ts) in zip(sorted(db_participants), baseline_raw):
    if par1==par2:
        rows.append([gr, par1, ts.T])
    else:
        print("skipped {}".format(par1))
        break

In [107]:
baseline_raw_df = pd.DataFrame(data=rows,columns='group,participant,signal_2hz'.split(','))


baseline_raw_df[baseline_raw_df.signal_2hz.map(lambda l: l.shape[-1]<50)]

,group,participant,signal_2hz


In [108]:
baseline_raw_df.head()


,group,participant,signal_2hz
0,1001,101,"[[3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5,..."
1,1001,102,"[[3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5,..."
2,1001,103,"[[3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5,..."
3,1003,108,"[[4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0,..."
4,1003,109,"[[4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0,..."


In [109]:
baseline_raw_df['remez_01_015']=baseline_raw_df.signal_2hz.map(lowpass)

In [110]:
baseline_raw_df.to_json(path.join(data_dir,'baseline_raw.json'))